In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import savefig
import re
import time
import os

In [11]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [13]:
project = 'other' # or 'other' for uCLIMB, etc. 

dataFolder = './SavedDataUCLIMB/'

In [14]:
def getSubjectNumber(filename):
    subjNumberRegex = re.compile('''
    # filename will be something like 'OpenBCI-RAW-28211_SBR_Pre1.txt'
    # Regex looks for a 5-digit string in the filename
    # Separators are inconsistent, so parsing only by continuous digits
    (\d\d\d\d\d)
    ''', re.VERBOSE)

    extractSubjNumber = subjNumberRegex.findall(filename)
    subjNumber = extractSubjNumber[0]

    return subjNumber

In [15]:
def getBaGELSSubjectNumber(filename):
    subjNumberRegex = re.compile('''
    # filename will be something like 'OpenBCI-RAW-BaGELS_sEBR_9001.txt'
    # Regex looks for a 4-digit string in the filename
    # Separators are inconsistent, so parsing only by continuous digits
    (\d\d\d\d)
    ''', re.VERBOSE)

    extractSubjNumber = subjNumberRegex.findall(filename)
    subjNumber = extractSubjNumber[0]

    return subjNumber

In [16]:
def blinkDetect(DataFrame):
    '''
    Preprocesses time series data and counts blink individually for channels one and two.
    '''

    halfSecond = int(np.max(raw[0])/4) # rolling window width used for mean estimation
    samplingRate = np.max(raw[0]) # extracting sampling rate from max index value   
    lowerBoundary = samplingRate*30 # lower cutoff for sEBR recording (exclude first 30 seconds)
    upperBoundary = len(raw[1])-(lowerBoundary) # upper cutoff for sEBR recording (exclude last 30 seconds)


    # Making time series mean stationary to account for signal drift on a second-by-second basis
    raw1RollingMean = raw[1].rolling(window=samplingRate).mean() # computing rolling window average
    channelOneStationary = raw[1] - raw1RollingMean # subtracting TS mean from each datapoint
    
    raw2RollingMean = raw[2].rolling(window=samplingRate).mean()
    channelTwoStationary = raw[2] - raw1RollingMean
    
    # Discarding first and last 30 seconds of data
    channelOneRelevant = channelOneStationary[lowerBoundary:upperBoundary] # dropping first and last 30 seconds
    
    channelTwoRelevant = channelTwoStationary[lowerBoundary:upperBoundary]
    
    # fft to remove unwanted frequencies
    channelOneNorm_fft = np.fft.fft(channelOneRelevant) # taking TS to freq space
    channelOneNorm_fft[0:1] = 0 # removing low frequencies
    channelOneNorm_fft[2500:] = 0 # removing high frequencies
    channelOneNorm_ifft = np.fft.ifft(channelOneNorm_fft) # taking TS back to native space

    channelTwoNorm_fft = np.fft.fft(channelTwoRelevant)
    channelTwoNorm_fft[0:1] = 0
    channelTwoNorm_fft[2500:] = 0
    channelTwoNorm_ifft = np.fft.ifft(channelTwoNorm_fft)

    channelOneTwo_ifft = pd.DataFrame(channelOneNorm_ifft) # creating Pandas DF to use rolling window later
    channelOneTwo_ifft[1] = channelTwoNorm_ifft # adding channel two data into DF

    # Algorithm: Using Channels one and two to find blinks, while discarding first and last 30 seconds of data
    r = channelOneTwo_ifft.rolling(window=50).var() # rolling window using variance 
    
    channelOneThreshold = np.mean(r[0][lowerBoundary:upperBoundary])*2.5 # setting threshold for blink
    channelTwoThreshold = np.mean(r[1][lowerBoundary:upperBoundary])*2.5 # TODO: Chat with AS/CP about arbitrary thresholding
    
    channelOneBlinks = 0
    channelTwoBlinks = 0

    for i in range(len(channelOneTwo_ifft)):
        if r[0][i] > channelOneThreshold:
            channelOneBlinks += 1
    for i in range(len(channelOneTwo_ifft)):
        if r[1][i] > channelTwoThreshold:
            channelTwoBlinks += 1
    
    channelOneBlinksRounded = round(channelOneBlinks/(samplingRate/4)) # 
    channelTwoBlinksRounded = round(channelTwoBlinks/(samplingRate/4))
    return channelOneBlinksRounded, channelTwoBlinksRounded

In [17]:
def plotTimeSeries(DataFrame, subjectNo):
    '''
    Plots preprocessed time series data that is used to count blinks. Saves plots to a new "TSplots" directory in the root folder.
    '''
    halfSecond = int(np.max(raw[0])/4) # rolling window width used for mean estimation
    samplingRate = np.max(raw[0]) # extracting sampling rate from max index value   
    lowerBoundary = samplingRate*30 # lower cutoff for sEBR recording (exclude first 30 seconds)
    upperBoundary = len(raw[1])-(lowerBoundary) # upper cutoff for sEBR recording (exclude last 30 seconds)
    
    if not os.path.exists('./TSplots'):
        os.makedirs('./TSplots')
    
    raw1RollingMean = raw[1].rolling(window=samplingRate).mean() # computing rolling window average
    channelOneStationary = pd.DataFrame(raw[1] - raw1RollingMean) # subtracting TS mean from each datapoint
    
    raw2RollingMean = raw[2].rolling(window=samplingRate).mean()
    channelTwoStationary = pd.DataFrame(raw[2] - raw1RollingMean)
    
    # TODO: plot denoised data
    
    channelOneStationary[lowerBoundary:upperBoundary].plot(figsize=(20,10))   
    savefig('./TSplots/' + subjectNo + '_channelOne.png')
    plt.close()
    
    channelTwoStationary[lowerBoundary:upperBoundary].plot(figsize=(20,10))   
    savefig('./TSplots/' + subjectNo + '_channelTwo.png')
    plt.close()

In [18]:
%%capture
# Counting blinks for all .txt files in data directory. 
# Creates .csv with subject number and blinks for each channel, saved in root directory. 
subjectNumberList = []
channelOneBlinkList = []
channelTwoBlinkList = []
for folderName, subfolders, filenames in os.walk(dataFolder):

    for file in filenames:
        if file.endswith('.py'):
            pass
        elif file.endswith('.ipynb'):
            pass
        elif file.endswith('.txt'):
            print(file) # Trobleshoot
            raw = pd.read_table(dataFolder + file, sep = ',', skiprows=6, header=None)

            if project == 'BaGELS':
                subjectNo = getBaGELSSubjectNumber(file)
                subjectNumberList.append(subjectNo)
            else:
                subjectNo = getSubjectNumber(file)
                subjectNumberList.append(subjectNo)
            
            plotTimeSeries(raw, subjectNo)
            
            channelOneBlinks, channelTwoBlinks = blinkDetect(raw)
            channelOneBlinkList.append(channelOneBlinks)
            channelTwoBlinkList.append(channelTwoBlinks)

currentDateTime = time.strftime("%m.%d.%Y_%H.%M%p")
filename = './getBlinks_output_' + currentDateTime + '.csv'
blinkOutput = open(filename, 'w')

blinkOutput.write('SubjectNo' + ',' + 'channelOneBlinks' + ',' + 'channelTwoBlinks' + '\n')
zipped = zip(subjectNumberList, channelOneBlinkList, channelTwoBlinkList)
for i, j, k in zipped:
    blinkOutput.write(str(i) + ',' + str(j) + ',' + str(k) + '\n')
blinkOutput.close()

print('sEBR count finished. Please check root folder for output.')